## Linear Regression

In [3]:
using DataFrames
using CSV
using Plots
using Lathe
using GLM
using Statistics
using StatsPlots
using MLBase

In [ ]:
# Enable printing of 1000 columns
ENV["COLUMNS"] = 1000

### Import Data

In [4]:
df = DataFrame(CSV.File("../data/diabetes_matrix.csv"))

,Column1,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64
1,0,13,3.666,1,0,1,0,0
2,1,18,1.2885,1,0,1,0,0
3,2,20,1.0508,1,1,0,0,0
4,3,21,14.2664,2,0,1,0,0
5,4,24,0.5124,1,0,1,0,0
6,5,25,3.5466,1,0,1,0,0
7,6,28,1.1224,1,1,0,0,0
8,7,35,5.3757,1,1,0,0,0
9,8,37,1.1397,1,0,1,0,0


In [18]:
names(df)

931-element Array{Symbol,1}:
 :Column1
 :SUBJECT_ID
 :LOS
 :HOSPITALIZATION
 :ELECTIVE
 :EMERGENCY
 :NEWBORN
 :URGENT
 :Government
 :Medicaid
 :Medicare
 :Private
 Symbol("Self Pay")
 ⋮
 Symbol("HISPANIC/LATINO - COLOMBIAN")
 Symbol("WHITE - BRAZILIAN")
 Symbol("NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER")
 Symbol("HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)")
 Symbol("ASIAN - JAPANESE")
 Symbol("ASIAN - THAI")
 Symbol("HISPANIC/LATINO - HONDURAN")
 Symbol("HISPANIC/LATINO - CUBAN")
 Symbol("MIDDLE EASTERN")
 Symbol("ASIAN - OTHER")
 Symbol("HISPANIC/LATINO - MEXICAN")
 Symbol("AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE")

### Fix Column Names 

In [18]:
colnames = Symbol[]
for i in string.(names(df))
    push!(colnames, Symbol(replace(replace(replace(replace(
                        replace(strip(i)," " => "_"),"-" => "_"), "/" => "_or_"),"(" => "_"), ")" => "_")))
end

#rename!(df, colnames);

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Symbol
Closest candidates are:
  convert(::Type{S}, !Matched::T) where {S, T<:(Union{CategoricalString{R}, CategoricalValue{T,R} where T} where R)} at /Users/rayeliu/.julia/packages/CategoricalArrays/dmrjI/src/value.jl:103
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171
  Symbol(::String) at boot.jl:436
  ...

In [17]:
println(colnames)

[:Column1, :SUBJECT_ID, :LOS, :HOSPITALIZATION, :ELECTIVE, :EMERGENCY, :NEWBORN, :URGENT, :Government, :Medicaid, :Medicare, :Private, :Self_Pay, :DIVORCED, :LIFE_PARTNER, :MARRIED, :NAN, :SEPARATED, :SINGLE, :UNKNOWN, :WIDOWED, :GENDER, Symbol("003"), Symbol("004"), Symbol("005"), Symbol("007"), Symbol("008"), Symbol("009"), Symbol("010"), Symbol("011"), Symbol("012"), Symbol("013"), Symbol("014"), Symbol("018"), Symbol("027"), Symbol("031"), Symbol("032"), Symbol("034"), Symbol("035"), Symbol("038"), Symbol("039"), Symbol("040"), Symbol("041"), Symbol("042"), Symbol("046"), Symbol("047"), Symbol("048"), Symbol("049"), Symbol("052"), Symbol("053"), Symbol("054"), Symbol("057"), Symbol("058"), Symbol("066"), Symbol("070"), Symbol("075"), Symbol("078"), Symbol("079"), Symbol("082"), Symbol("086"), Symbol("088"), Symbol("094"), Symbol("096"), Symbol("097"), Symbol("110"), Symbol("111"), Symbol("112"), Symbol("117"), Symbol("120"), Symbol("121"), Symbol("127"), Symbol("130"), Symbol("131"

Symbol("923"), Symbol("924"), Symbol("926"), Symbol("927"), Symbol("933"), Symbol("934"), Symbol("935"), Symbol("936"), Symbol("937"), Symbol("939"), Symbol("941"), Symbol("942"), Symbol("944"), Symbol("945"), Symbol("947"), Symbol("948"), Symbol("950"), Symbol("951"), Symbol("952"), Symbol("953"), Symbol("954"), Symbol("955"), Symbol("956"), Symbol("958"), Symbol("959"), Symbol("961"), Symbol("962"), Symbol("964"), Symbol("965"), Symbol("966"), Symbol("967"), Symbol("968"), Symbol("969"), Symbol("970"), Symbol("971"), Symbol("972"), Symbol("974"), Symbol("975"), Symbol("977"), Symbol("983"), Symbol("987"), Symbol("989"), Symbol("990"), Symbol("991"), Symbol("992"), Symbol("994"), Symbol("995"), Symbol("996"), Symbol("997"), Symbol("998"), Symbol("999"), :E812, :E813, :E814, :E815, :E816, :E817, :E818, :E819, :E821, :E822, :E823, :E824, :E825, :E826, :E838, :E848, :E849, :E850, :E851, :E852, :E853, :E854, :E855, :E857, :E858, :E866, :E869, :E870, :E871, :E874, :E876, :E878, :E879, :E88

In [ ]:
for c in colnames:
    println()

In [12]:
rename!(df, colnames);

MethodError: MethodError: no method matching iterate(::Symbol)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:603
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:603
  iterate(!Matched::ExponentialBackOff) at error.jl:253
  ...

In [ ]:
println(size(df))
describe(df)

In [ ]:
scaled_feature = Lathe.preprocess.OneHotEncode(df,:Status)
select!(df, Not([:Status,:Country]))
first(df,5)

In [ ]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75)

In [ ]:
fm = @formula(Life_expectancy ~ Adult_Mortality)
linearRegressor = lm(fm, train)